In [43]:
import numpy as np
import pandas as pd
import random
import time
from sklearn.model_selection import RepeatedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import linalg
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
import requests

In [44]:
df = pd.read_excel (r'/Users/qizhe/Desktop/School/RA/Email-Eu-Core/Labels.xlsx',header = None)
y = df.to_numpy()

In [45]:
df2 = pd.read_excel (r'/Users/qizhe/Desktop/School/RA/Email-Eu-Core/Core.xlsx',header = None)
Edge = df2.to_numpy()

In [46]:
A = np.zeros((1005,1005))
for i in range (0,25571):
    A[Edge[i,0],Edge[i,1]]=1

In [53]:
def EncoderLDA(A):
#Split Data
    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=5)
    for train_index, test_index in rkf.split(A):
         #print("TRAIN:", train_index, "TEST:", test_index)
         X_train, X_test = A[train_index], A[test_index]
         y_train, y_test = y[train_index], y[test_index]
         X1 = X_train.transpose()
         X_train, X_test = X1[train_index], X1[test_index]
#ASE*LDA
    begin1 = time.time()
    U, s, VT = np.linalg.svd(A)
    end1 = time.time()
    newx=([])
    dmax=50
    tmp = np.zeros([dmax,1])
    for d in range (1,dmax):
        newx=np.concatenate(U[:,0:d]*s[0:d]**0.5)
        newx=np.reshape(newx,(n,d))
        clf2 = LinearDiscriminantAnalysis()
        newx_train, newx_test =newx[train_index], newx[test_index]
        clf2.fit(newx_train, y_train.ravel())
        tmp[d,0]=clf2.score(newx_test, y_test.ravel())
    ASELDAACC=tmp.max()
    ASELDATIME=end1-begin1
    eigenvalues=s**2
    Top=eigenvalues[:10]
#nk,w,Z
    begin5 = time.time()
    global k
    nk = np.zeros((1,k))
    for i in range(0,len(y_train)-1):
        nk[0,int(y_train[i,0]-1)]=nk[0,int(y_train[i,0]-1)]+1
    w = np.zeros((int(np.size(y_train)),k))
    for i in range(0,int(np.size(y_train))):
        k=int(y_train[i])
        w[i][k-1]=1/nk[0,k-1]
    z_train=np.matmul(X_train,w)
    z_test = np.matmul(X_test,w)
    end5 = time.time()
    zTIME=end5 - begin5
#AEE*KNN
    begin2 = time.time()
    neigh = KNeighborsClassifier(weights='uniform',metric='euclidean')
    neigh.fit(z_train, y_train.ravel())
    AEEKNNACC = neigh.score(z_test, y_test)
    end2 = time.time()
    AEEKNNTIME=end2-begin2
#AEE*LDA
    begin3 = time.time()
    clf4 = LinearDiscriminantAnalysis()
    clf4.fit(z_train,y_train.ravel())
    AEELDAACC = clf4.score(z_test, y_test.ravel())
    end3 = time.time()
    AEELDATIME=end3-begin3
#Get result
    Acc=np.array([ASELDAACC,AEEKNNACC,AEELDAACC])
    Time=np.array([ASELDATIME,AEEKNNTIME+zTIME,AEELDATIME+zTIME])
    print(f"Accuracy for each method: {Acc}")
    print(f"Time for each method: {Time}")

In [54]:
n=1005
k=int(y.max())
EncoderLDA(A)

Accuracy for each method: [0.59701493 0.6318408  0.66666667]
Time for each method: [0.49791002 0.02159619 0.01113009]
